REA Differential Equations Notebook

* Page 1A

Tutor:
* Anthropic's AI, Claude

###Concise summary of key approaches for error analysis in ODE solutions.


1. Analytical vs Numerical Comparison (when possible):
   - Compare against exact solution if available
   - Calculate both absolute and relative errors
   - Focus on maximum error and RMS error

2. Method Validation:
   - Use multiple numerical methods (e.g., RK45, DOP853)
   - Compare solutions across different tolerances
   - Look for convergence as step size decreases

3. Stability Analysis:
   - Check sensitivity to initial conditions
   - Verify solution remains bounded
   - Monitor energy conservation for relevant systems

4. Error Estimation:
   - Use embedded error estimators in adaptive methods
   - Compare solutions at different tolerances
   - Calculate local and global error estimates

5. Practical Considerations:
   - Balance computational cost vs accuracy
   - Consider problem-specific requirements (conservation laws, positivity)
   - Document assumptions and limitations



In [ ]:
"""
Abstract:
This module provides a comprehensive framework for analyzing errors in numerical solutions of Ordinary
Differential Equations (ODEs). It implements multiple error analysis strategies including:
- Comparison with analytical solutions
- Cross-validation between different numerical methods
- Convergence analysis across various tolerance levels
- Stability assessment through perturbation analysis
- Statistical error metrics computation

The framework is designed to help researchers and engineers validate their ODE solutions and make
informed decisions about method selection and parameter choices for specific problems.

Authors: Claude
Date: November 2024
"""

import numpy as np
from scipy.integrate import solve_ivp

def analyze_ode_errors(f, y0, t_span, t_eval, exact_solution=None):
    """
    Comprehensive ODE error analysis framework that evaluates solution accuracy and stability
    across different numerical methods and tolerance levels.

    Parameters:
    -----------
    f : callable
        Right-hand side of ODE system, must have signature f(t, y)
        where t is a scalar and y is a numpy array
    y0 : array_like
        Initial conditions of the system
    t_span : tuple
        Tuple defining the integration interval (t0, tf)
    t_eval : array_like
        Time points at which to evaluate the solution
    exact_solution : callable, optional
        Analytical solution function with signature exact_solution(t)
        Returns exact values at time points t

    Returns:
    --------
    results : dict
        Dictionary containing numerical solutions for each method-tolerance combination
        Keys are formatted as "method_tolerance" (e.g., "RK45_1e-3")
    errors : dict
        Dictionary containing various error metrics:
        - absolute_* : Absolute errors vs exact solution
        - relative_* : Relative errors vs exact solution
        - method_difference_* : Differences between numerical methods
        - tolerance_difference_* : Solution changes across tolerances
    """
    # Define solution methods and tolerance levels for comprehensive analysis
    methods = ['RK45', 'DOP853']  # RK45: Runge-Kutta 5(4), DOP853: Dormand-Prince 8(5,3)
    tolerances = [1e-3, 1e-6, 1e-9]  # Varying precision levels

    # Dictionary to store numerical solutions
    results = {}

    # Compute solutions for each method-tolerance combination
    for method in methods:
        for tol in tolerances:
            # Solve IVP with current parameters
            sol = solve_ivp(
                f, t_span, y0,
                method=method,
                rtol=tol,  # Relative tolerance
                atol=tol/10,  # Absolute tolerance (typically smaller than rtol)
                t_eval=t_eval  # Specific time points for solution evaluation
            )
            results[f"{method}_{tol}"] = sol.y[0]

    # Initialize error analysis dictionary
    errors = {}

    # 1. Analytical Error Analysis (if exact solution available)
    if exact_solution is not None:
        exact_values = exact_solution(t_eval)
        for key, result in results.items():
            # Compute absolute errors
            errors[f"absolute_{key}"] = np.abs(result - exact_values)
            # Compute relative errors (avoiding division by zero)
            errors[f"relative_{key}"] = np.abs((result - exact_values)/exact_values)

    # 2. Method Comparison Analysis
    # Compare solutions between different numerical methods at same tolerance
    for tol in tolerances:
        method1, method2 = methods
        diff = np.abs(results[f"{method1}_{tol}"] - results[f"{method2}_{tol}"])
        errors[f"method_difference_{tol}"] = diff

    # 3. Tolerance Convergence Analysis
    # Analyze how solutions change with decreasing tolerance
    for method in methods:
        for tol in tolerances[:-1]:
            current = results[f"{method}_{tol}"]
            next_tol = tolerances[tolerances.index(tol) + 1]
            next_result = results[f"{method}_{next_tol}"]
            errors[f"tolerance_difference_{method}_{tol}"] = np.abs(current - next_result)

    return results, errors

def error_metrics(errors):
    """
    Calculate statistical metrics for each type of error.

    Parameters:
    -----------
    errors : dict
        Dictionary containing different types of errors as numpy arrays

    Returns:
    --------
    metrics : dict
        Dictionary containing statistical metrics for each error type:
        - max: Maximum error value
        - mean: Mean error value
        - rms: Root Mean Square error

    Notes:
    ------
    RMS error is particularly useful as it penalizes larger errors more heavily
    than smaller ones due to the squaring operation.
    """
    metrics = {}
    for error_type, error_values in errors.items():
        metrics[f"{error_type}_max"] = np.max(error_values)  # Maximum error
        metrics[f"{error_type}_mean"] = np.mean(error_values)  # Average error
        metrics[f"{error_type}_rms"] = np.sqrt(np.mean(error_values**2))  # RMS error
    return metrics